In [1]:
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [3]:
#you can download the pretrained weights from the following link 
#https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
#or you can find the detailed documentation https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

from keras.models import model_from_json
model.load_weights('vgg_face_weights.h5')

In [5]:
def preprocess_image(image_path):
    #img = load_img(image_path, target_size=(224, 224))
    #img = img_to_array(img)
    #img = np.expand_dims(image_path, axis=0)
    img = preprocess_input(image_path)
    return img

In [6]:
def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [7]:
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [11]:
import cv2
import sys
import glob

faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')
video_capture = cv2.VideoCapture(0)

threshold=0.2
(width, height) = (224, 224)
idx=0
                 
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    faces = faceCascade.detectMultiScale(
        frame,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(224, 224)
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        rrec = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        face = frame[y:y + h, x:x + w] 
        face_resize = cv2.resize(face, (width, height))
        face_resize = face_resize.reshape(1,224,224,3)
        img1_representation = vgg_face_descriptor.predict(preprocess_image(face_resize))[0,:]
        
        images=glob.glob('dataset/*.jpg')
        cosines=[]
        faces=[]
        for i in range(len(images)):
            img2=cv2.imread(images[i])
            img2=cv2.resize(img2, (width, height))
            img2=img2.reshape(1,224,224,3)
            
            img2_representation = vgg_face_descriptor.predict(preprocess_image(img2))[0,:]

            cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
            cosines.append(cosine_similarity)
        
        minimum = cosines.index(min(cosines))
        #percent = (0.4-float(cosines[minimum]))*100/0.4
        #print(str(percent)+'%')
        print(cosines[minimum])
        print(images[minimum])
        print('-----------------------------')
        
        if float(cosines[minimum])<0.2:
            text=str(images[minimum])
            
        if float(cosines[minimum])>0.2:
            text='not found'
            write_name = 'dataset/new_face.'+str(idx)+'.jpg'
            cv2.imwrite(write_name, face)
            idx+=1
        
        cv2.putText(rrec,text,(x,y), cv2.FONT_HERSHEY_SIMPLEX, 1,(167,34,56),1,cv2.LINE_AA)
        
        
        
        
        
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

0.2953547239303589
dataset/123.jpg
-----------------------------
0.03470110893249512
dataset/new_face.0.jpg
-----------------------------
0.012829363346099854
dataset/new_face.0.jpg
-----------------------------
0.020385563373565674
dataset/new_face.0.jpg
-----------------------------
0.03111577033996582
dataset/new_face.0.jpg
-----------------------------
0.040417492389678955
dataset/new_face.0.jpg
-----------------------------
0.02755528688430786
dataset/new_face.0.jpg
-----------------------------
0.10518282651901245
dataset/123.jpg
-----------------------------
0.09411978721618652
dataset/123.jpg
-----------------------------
0.10860228538513184
dataset/123.jpg
-----------------------------
0.09581100940704346
dataset/123.jpg
-----------------------------
0.09093517065048218
dataset/123.jpg
-----------------------------
0.07239365577697754
dataset/123.jpg
-----------------------------
0.06612032651901245
dataset/123.jpg
-----------------------------


In [ ]:
import glob
import cv2
epsilon=0.2
images=glob.glob('dataset/*.jpg')
(width, height) = (224, 224)

def func(img1):
    img1_representation = vgg_face_descriptor.predict(preprocess_image(img1))[0,:]
    
    cosines=[]
    faces=[]
    for i in range(len(images)):
        img2=cv2.imread(images[i])
        img2=cv2.resize(img2, (width, height))
        img2.shape
        img2=img2.reshape(1,224,224,3)
        img2_representation = vgg_face_descriptor.predict(preprocess_image(img2))[0,:]

        cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
        cosines.append(cosine_similarity)
    print(cosines)
    print(images)